In [1]:
import yaml 
with open("/root/graduation_thetis/causal-bert-pytorch/run/result/[1202]seed_everything_appliance_sharpness_ave_none.yaml",'r') as yml:
    yaml_data = yaml.safe_load(yml)
print(yaml_data)

{'exp1': {'ATE': -0.12615819592841013, 'batch_size': 32, 'desc': 'con:sharpness_ave, treat: 0.8, confound: 10.0, aug None', 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 42}, 'exp10': {'ATE': -0.10450058692676101, 'batch_size': 32, 'desc': 'con:sharpness_ave, treat: 0.8, confound: 10.0, aug None', 'epochs': 3, 'model_name': 'timm/eva02_tiny_patch14_224.mim_in22k', 'seed': 2021}, 'exp100': {'ATE': 0.03370459633044437, 'batch_size': 32, 'desc': 'con:sharpness_ave, treat: 0.8, confound: 10.0, aug None', 'epochs': 3, 'model_name': 'timm/vit_base_patch32_clip_224.laion2b_ft_in12k_in1k', 'seed': 4041}, 'exp101': {'ATE': -0.0945344963600554, 'batch_size': 32, 'desc': 'con:sharpness_ave, treat: 0.8, confound: 10.0, aug None', 'epochs': 5, 'model_name': 'timm/vit_base_patch32_clip_224.laion2b_ft_in12k_in1k', 'seed': 42}, 'exp102': {'ATE': -0.09008204303006319, 'batch_size': 32, 'desc': 'con:sharpness_ave, treat: 0.8, confound: 10.0, aug None', 'epochs': 5, 'model_n

In [2]:
import numpy as np
eva_ate = []
vit_ate = []
eff_ate = []
for k,v in yaml_data.items():
    if v['model_name'] == 'timm/eva02_tiny_patch14_224.mim_in22k':
        eva_ate.append(v['ATE'])
    if v['model_name'] == 'timm/efficientvit_m2.r224_in1k':
        eff_ate.append(v['ATE'])
    if v['model_name'] == 'timm/vit_base_patch32_clip_224.laion2b_ft_in12k_in1k':
        vit_ate.append(v['ATE'])

print("eva_ate", np.mean(eva_ate),np.std(eva_ate))
print("vit_ate", np.mean(vit_ate),np.std(vit_ate))
print("eff_ate", np.mean(eff_ate),np.std(eff_ate))

eva_ate -0.056083535209342304 0.03364998191732666
vit_ate -0.04556121214313324 0.052220201328134534
eff_ate -0.1422582667710035 0.08486035404372319


In [3]:
import numpy as np
for epoch in [3,5,10]:
        eva_ate, vit_ate, eff_ate = [], [], []
        for k, v in yaml_data.items():
            if v["epochs"] != epoch:
                continue
            if v['model_name'] == 'timm/eva02_tiny_patch14_224.mim_in22k':
                eva_ate.append(v['ATE'])
            elif v['model_name'] == 'timm/efficientvit_m2.r224_in1k':
                eff_ate.append(v['ATE'])
            elif v['model_name'] == 'timm/vit_base_patch32_clip_224.laion2b_ft_in12k_in1k':
                vit_ate.append(v['ATE'])

        # ATEの統計量を計算
        eva_stats = f"eva_ate: mean={np.mean(eva_ate):.4f}, std={np.std(eva_ate):.4f}"
        vit_stats = f"vit_ate: mean={np.mean(vit_ate):.4f}, std={np.std(vit_ate):.4f}"
        eff_stats = f"eff_ate: mean={np.mean(eff_ate):.4f}, std={np.std(eff_ate):.4f}"
        print("eva_ate", np.mean(eva_ate),np.std(eva_ate))
        print("vit_ate", np.mean(vit_ate),np.std(vit_ate))
        print("eff_ate", np.mean(eff_ate),np.std(eff_ate))


eva_ate -0.06402207376521643 0.038995490149490886
vit_ate -0.04073008572150494 0.05730979056194454
eff_ate -0.1498782151891183 0.0898002888239778
eva_ate -0.048144996653468164 0.024858671027783878
vit_ate -0.05522346498638987 0.03836646187406765
eff_ate -0.13463831835288867 0.07888181207472861
eva_ate nan nan
vit_ate nan nan
eff_ate nan nan


/root/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/root/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/root/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/root/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/root/.venv/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [6]:
from collections import defaultdict
import numpy as np
import pandas as pd
def ATE_unadjusted(T, Y):
    x = defaultdict(list)
    for t, y in zip(T, Y):
        x[t].append(y)
    T0 = np.mean(x[0])
    T1 = np.mean(x[1])
    return T0 - T1

def ATE_adjusted(C, T, Y):
    x = defaultdict(list)
    for c, t, y in zip(C, T, Y):
        x[c, t].append(y)

    C0_ATE = np.mean(x[0,0]) - np.mean(x[0,1])
    C1_ATE = np.mean(x[1,0]) - np.mean(x[1,1])
    return np.mean([C0_ATE, C1_ATE])


if __name__ == "__main__":
    df = pd.read_csv("/root/graduation_thetis/causal-bert-pytorch/input/modelinput/Appliances_preprocess_sharpness_ave_t0.8c10.0_1202.csv")
    treatment = "light_or_dark"
    confounder = "sharpness_ave"
    outcome = "outcome"
    T_proxy = "T_proxy"
    print("ATE_unadjusted: ", ATE_unadjusted(df[T_proxy], df[outcome]))
    print("ATE_adjusted: ", ATE_adjusted(df[confounder], df[treatment],df[outcome]))

ATE_unadjusted:  -0.24485774896112988
ATE_adjusted:  -0.13700807618489003


In [6]:
df[confounder].value_counts()

contains_text
0    4397
1     511
Name: count, dtype: int64